# Skin Effect in a High-Voltage Cable
In this notebook, a section of a high-voltage cable is simulated (taken from the COMSOL project of EE4625 - High Voltage Cable Systems). The simulation is magnetostatics including a term for induced eddy currents. 

## Setup

## Theory - Eddy Currents
To include eddy currents, the differential equations for magnetostatics must be extended with an additional term. Initially we write
$$ \nabla \times (\nu \nabla \times \mathbf{A}) = \mathbf{J} $$
Assuming that there is only current in the $z$ direction, and that $\mathbf{J}$ is now a combination of conduction current and induced current, we find
$$ -\nabla \cdot (\nu \nabla A_z) = J_z + \sigma E_z $$
The electric field is defined as
$$ \mathbf{E} = -\nabla\phi - \frac{\partial \mathbf{A}}{\partial t} $$
We will now make two assumptions: first, the electrical potential is zero; second, the quantities are time-harmonic. The differential equation then becomes
$$ \nabla \cdot (\nu \nabla A_z) - j \sigma\omega A_z = -J_z $$

## Theory - Variational Form
This differential equation can be written in the variational form as follows
$$ \int_\Omega \nu \nabla A_z \cdot \nabla v d\Omega + j\omega \int_\Omega \sigma A_z \cdot v d\Omega = \int_\Omega J_z \cdot v d\Omega + \oint_{\Gamma_N} \nu \frac{\partial A_z}{\partial \mathbf{n}} \cdot v d\Gamma \qquad v \in V_0 $$
We will only use Dirichlet boundary conditions, and a symmetry condition for this problem, so the boundary integral over $\Gamma_N$ becomes zero.
$$ \int_\Omega \nu \nabla A_z \cdot \nabla v d\Omega + j\omega \int_\Omega \sigma A_z \cdot v d\Omega = \int_\Omega J_z \cdot v d\Omega \qquad v \in V_0 $$

## Theory - Field-Circuit Coupling
If this formulation is now solved, the following phenomenon is observed. When the frequency is increased, the eddy current term will _reduce_ the total current. If the integral of the total current $J = J_z - j\sigma\omega A_z$ is taken, this will tend to zero as the frequency increases. This obviously cannot be true, as the total current through the cable must stay the same. Instead, the voltage across it will change.

To resolve this, an additional equation is added to the system to ensure that the current is determined by the external circuit. For example, in COMSOL, this is done using a current-driven "coil", with the following equation.
$$ I_{coil} = \iint_{\text{coil}} \mathbf{J} \cdot \mathbf{e}_{\text{coil}} d\Omega $$
where $\mathbf{e}_{\text{coil}}$ is the unit vector in the direction of the coil (i.e., out of the screen).

Using the previously defined total current density (which is conduction current plus the eddy current term),
$$ I_{coil} = \iint_{\text{coil}} (J_z - j\sigma\omega A_z) d\Omega $$
Here, $J_z$ is the source term, we assume this to be of the form $J_z = \frac{I_c}{A}$, where $I_c$ is an artifical variable (which will be added to the vector of unknowns) denoting the source current, and $A$ is the cross-section of the coil. The equation then becomes
$$ I_{coil} = I_c - j\sigma\omega \iint_{\text{coil}} A_z d\Omega $$
The integral can be expressed in terms of the mesh elements by
$$ I_{coil} \approx I_c -j\sigma\omega \sum_{k = 1}^{N_{el}} \operatorname{area}(e_k) \cdot \frac{1}{3} \left[ A_{k,1} + A_{k,2} + A_{k,3} \right] $$
where $A_{k,i}$ is the magnetic potential $A_z$ in local node $i$ of element $k$.

## Theory - Linear System Modification
The linear system is then modified in the following way:
$$ A \mathbf{c} = \mathbf{f} \rightarrow \tilde{A} \tilde{c} = \tilde{f}$$
where the matrices are as follows.
$$\tilde{A} = \begin{bmatrix}
    A   & \mathbf{a}_1 \\
    \mathbf{a}_2 & a_3 \\
\end{bmatrix} \qquad \tilde{c} = \begin{bmatrix} 
    \mathbf{c} \\ I_c
\end{bmatrix} \qquad \tilde{f} = \begin{bmatrix}
    \mathbf{f} \\ I_{coil}
\end{bmatrix} $$
The coefficients $\mathbf{a}_1$, $\mathbf{a}_2$, $a_3$ are determined from the field-circuit equation given above. The source term $\mathbf{f}$ is made zero to reflect that the current density is now determined by the unknown $I_c$.
\begin{align*}
    \mathbf{a}_1 & = \\
    \mathbf{a}_2 & = \\
    a_3 & = \\
\end{align*}

## Theory - AC Resistance
It is now of interest to calculate the AC resistance of the cable. This can be done by integrating the current density
$$ R_{AC} = \frac{1}{I_{coil}^2} \iint_{\text{coil}} \frac{|J|^2}{\sigma} d\Omega $$

# Include Packages

In [1]:
using LinearAlgebra;
using Plots;

using gmsh;

using WriteVTK;

# Geometry

In [2]:
ri = 19.1e-3;   # Cable inner radius
ro = 37.5e-3;   # Cable outer radius

lc1 = 2e-3;      # Mesh density at outside of cable
lc2 = 0.1e-3;   # Mesh density at the conductor edge

# Define Geometry in gmsh

In [ ]:
gmsh.finalize()
gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("cable_skin")

# Points
gmsh.model.geo.addPoint(0, 0, 0, lc1, 1);
gmsh.model.geo.addPoint(ri, 0, 0, lc2, 2);
gmsh.model.geo.addPoint(ro, 0, 0, lc1, 3);
gmsh.model.geo.addPoint(0, ri, 0, lc2, 4);
gmsh.model.geo.addPoint(0, ro, 0, lc1, 5);

# Lines
gmsh.model.geo.addLine(1, 2, 1);
gmsh.model.geo.addLine(2, 3, 2);
gmsh.model.geo.addLine(1, 4, 3);
gmsh.model.geo.addLine(4, 5, 4);
gmsh.model.geo.addCircleArc(2, 1, 4, 5);
gmsh.model.geo.addCircleArc(3, 1, 5, 6);

# Surfaces
gmsh.model.geo.addCurveLoop([1, 5, -3], 1)
gmsh.model.geo.addCurveLoop([2, 6, -4, -5], 2)

gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.addPlaneSurface([2], 2)

# Physics
#  Dirichlet boundary on outside of cable
gmsh.model.addPhysicalGroup(0, [3, 5], 1)
gmsh.model.setPhysicalName(0, 1, "D1p")
gmsh.model.addPhysicalGroup(1, [6], 1)
gmsh.model.setPhysicalName(1, 1, "D1")

#  Neumann boundary on two insides
gmsh.model.addPhysicalGroup(0, [1, 2, 4], 2)
gmsh.model.setPhysicalName(0, 2, "N1p")
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4], 2)
gmsh.model.setPhysicalName(1, 2, "N1")

#  Material groups
gmsh.model.addPhysicalGroup(2, [1], 1)
gmsh.model.setPhysicalName(2, 1, "conductor")
gmsh.model.addPhysicalGroup(2, [2], 2)
gmsh.model.setPhysicalName(2, 2, "insulator")

# Generate mesh
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("cable_skin.msh")

# Load Geometry

In [3]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

In [4]:
# Structure for saving mesh data and passing it between functions
struct mesh_data
    nnodes                 # Number of nodes
    xnode                  # x-coordinates of nodes
    ynode                  # y-coordinates of nodes
    
    nelements              # Number of elements
    element_connectivity   # Array of elements connectivity
    e_group                # Material group of each element
end

# Load mesh data from gmsh: nodes and elements
function get_mesh_data()
    #..2/11 Get and sort the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    
    #..4/12 Get the mesh elements
    #..observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelements = length(element_ids[1])

    #..5/12 Create groups of elements for the subdomains
    #..for loop that creates a vector describing which physical group an element belongs to
    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
    e_group = zeros(1,nelements)
    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        G1 = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Conductor
        G2 = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Insulator
        if G1 == 3
            e_group[element_id] = 1;
        elseif G2 == 3
            e_group[element_id] = 2;
        end
    end
    
    return mesh_data(nnodes, xnode, ynode, nelements, element_connectivity, e_group)
end

get_mesh_data (generic function with 1 method)

In [5]:
gmsh.open("cable_skin.msh")
mshdata = get_mesh_data();

Info    : Reading 'cable_skin.msh'...
Info    : 13 entities
Info    : 7673 nodes
Info    : 15349 elements
Info    : Done reading 'cable_skin.msh'


# Define material & source functions

In [43]:
I = 1000;              # Current in the cable [A]

mu0 = 4*pi*1e-7;       # Permeability of vacuum
omega = 2*pi*50;       # Frequency of the time-harmonic current

sigma_cond = 3.69e7;   # Conductivity of the aluminium conductor

In [44]:
#..6/12 Define the source function and the reluctivity function
Jsource = I / (pi * ri*ri);
sourcefunction(group_id) = Jsource * (group_id == 1)
sourceperelement = map(sourcefunction, mshdata.e_group);

reluctivityfunction(group_id) = 1/ mu0;
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

conductivityfunction(group_id) = sigma_cond * (group_id == 1)
conductivityperelement = map(conductivityfunction, mshdata.e_group);

# FEM Solution

In [45]:
#..7/12 initialize global matrix A and global vector f
#..observe that for simplicity we use dense matrix here
N = mshdata.nnodes + 1;
A = zeros(Complex{Float64}, N, N)
f = zeros(Complex{Float64}, N, 1)

element_connectivity = mshdata.element_connectivity;
xnode = mshdata.xnode;
ynode = mshdata.ynode;

#..8/12 Perform a loop over the elements
for element_id in 1:mshdata.nelements
    #....retrieve global numbering of the local nodes of the current element
    node1_id = element_connectivity[1][3*(element_id-1)+1]
    node2_id = element_connectivity[1][3*(element_id-1)+2]
    node3_id = element_connectivity[1][3*(element_id-1)+3]

    #....retrieve the x and y coordinates of the local nodes of the current element
    xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
    ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

    #....compute surface area of the current element
    x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
    y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
    area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

    #....compute local vector contribution floc of the current element
    #floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

    #....compute local matrix contribution Aloc of the current element
    Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    Emat[3,:] .= 0;
    Bloc = area_id*reluctivityperelement[element_id]*(transpose(Emat)*Emat);
    Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

    #....add local contribution to f and A
    idx = element_connectivity[1][3 * (element_id - 1) .+ (1:3)];
    #f[I]   += floc;
    A[idx,idx] += (Bloc + Cloc);
    
    # Modify A and f with circuit equation
    f[N] = I / 4;
    if(mshdata.e_group[element_id] == 1)
        A[idx,end] += -4/(pi*ri*ri) * area_id/3 * [1;1;1];
        A[end,idx] += -1im * omega * conductivityperelement[element_id] * area_id/3 * [1;1;1];
    end
end
A[end,end] = 1;

#..9/12 Handle the boundary conditions
#..retrieve boundary nodes by loop over corner point and boundary edges
node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,5)
node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(1,6)
bnd_node_ids = union(node_ids1,node_ids2,node_ids3)
A[bnd_node_ids,:] .= 0;
A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
f[bnd_node_ids] .= 0;

#..10/12 Compute the numerical solution
u = A \ f;

# Post-processing

In [46]:
xel = zeros(mshdata.nelements);
yel = zeros(mshdata.nelements);
Jel = zeros(Complex{Float64}, mshdata.nelements);

Ic  = 0;
Rac = 0;
for element_id in 1:mshdata.nelements
    #....retrieve global numbering of the local nodes of the current element
    node1_id = element_connectivity[1][3*(element_id-1)+1]
    node2_id = element_connectivity[1][3*(element_id-1)+2]
    node3_id = element_connectivity[1][3*(element_id-1)+3]

    #....retrieve the x and y coordinates of the local nodes of the current element
    xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
    ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];
    
    #....compute surface area of the current element
    x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
    y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
    area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2
    
    xel[element_id] = (xnode1 + xnode2 + xnode3) / 3;
    yel[element_id] = (ynode1 + ynode2 + ynode3) / 3;
    if(mshdata.e_group[element_id] == 1)
        Jel[element_id] = 4*u[end] / (pi * ri*ri) - 1im * omega * conductivityperelement[element_id] * 1/3 * sum(u[[node1_id, node2_id, node3_id]]);
        Ic  = Ic + Jel[element_id] * area_id * 4;
        Rac = Rac + norm(Jel[element_id])^2 / conductivityperelement[element_id] * area_id;
    end
end

Rac = real(4 * Rac / Ic^2);

In [47]:
norm(Ic)

999.9948625218425

In [48]:
Rac

2.6756438807431834e-5

# Write to VTK

In [27]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, mshdata.element_connectivity[1][3*(i-1).+(1:3)]) for i = 1:mshdata.nelements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("cable_skin2.vtu", points, cells);

# Store data in the VTK file
vtkfile["A", VTKPointData()] = norm.(u[1:end-1]);
vtkfile["J", VTKCellData()]  = norm.(Jel);

# Save the file
outfiles = vtk_save(vtkfile);